## Amazon AgentCore Bedrock Code Interpreterを使用したエージェントベースのコード実行 - チュートリアル(Strands)

このチュートリアルでは、Pythonを使ってコードの実行によって回答を検証するAIエージェントを作成する方法を示します。LLMが生成したコードを実行するために、Amazon Bedrock AgentCore Code Interpreterを使用します。

このチュートリアルでは、AgentCore Bedrock Code Interpreterを使用して以下のことを実演します。
1. サンドボックス環境のセットアップ
2. ユーザーのクエリに基づいてコードを生成するストランドベースのエージェントの構成
3. Code Interpreterを使用してサンドボックス環境でコードを実行
4. 結果をユーザーに表示

## 前提条件
- Bedrock AgentCore Code Interpreterにアクセスできる AWS アカウント
- コードインタープリターリソースを作成および管理するための必要な IAM 権限
- 必要な Python パッケージがインストールされている (boto3、bedrock-agentcore、strands を含む)
- IAM ロールには Amazon Bedrock でモデルを呼び出す権限が必要
 - 米国オレゴン (us-west-2) リージョンで Claude 3.7 Sonnet モデルにアクセスできる (Strands SDK のデフォルトモデル)

## IAM 実行ロールには、以下の IAM ポリシーが添付されている必要があります

~~~ {
"Version": "2012-10-17",
"Statement": [
    {
        "Effect": "Allow",
        "Action": [
            "bedrock-agentcore:CreateCodeInterpreter",
            "bedrock-agentcore:StartCodeInterpreterSession",
            "bedrock-agentcore:InvokeCodeInterpreter",
            "bedrock-agentcore:StopCodeInterpreterSession",
            "bedrock-agentcore:DeleteCodeInterpreter",
            "bedrock-agentcore:ListCodeInterpreters",
            "bedrock-agentcore:GetCodeInterpreter"
        ],
        "Resource": "*"
    },
    {
        "Effect": "Allow",
        "Action": [
            "logs:CreateLogGroup",
            "logs:CreateLogStream",
            "logs:PutLogEvents"
        ],
        "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
    }
]
}

日本語訳:
"バージョン": "2012-10-17",
"ステートメント": [
    {
        "効果": "許可",
        "アクション": [
            "bedrock-agentcore:CreateCodeInterpreter",
            "bedrock-agentcore:StartCodeInterpreterSession",
            "bedrock-agentcore:InvokeCodeInterpreter",
            "bedrock-agentcore:StopCodeInterpreterSession",
            "bedrock-agentcore:DeleteCodeInterpreter",
            "bedrock-agentcore:ListCodeInterpreters",
            "bedrock-agentcore:GetCodeInterpreter"
        ],
        "リソース": "*"
    },
    {
        "効果": "許可",
        "アクション": [
            "logs:CreateLogGroup",
            "logs:CreateLogStream",
            "logs:PutLogEvents"
        ],
        "リソース": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
    }
]
~~~

## 動作の仕組み

コード実行サンドボックスは、コード インタプリター、シェル、ファイル システムを備えた分離された環境を作成することで、エージェントがユーザーのクエリを安全に処理できるようにします。大規模言語モデルがツールの選択を支援した後、コードがこのセッション内で実行され、その結果がユーザーまたはエージェントに合成のために返されます。

![architecture local](code-interpreter.png)

## 1. 環境のセットアップ

まず、必要なライブラリをインポートし、Code Interpreter セッションを初期化しましょう。

In [ ]:
!pip install --upgrade -r requirements.txt

In [4]:
from bedrock_agentcore.tools.code_interpreter_client import code_session
from strands import Agent, tool
import json
from typing import Dict, Any, List


## 2. システムプロンプトの定義
AI アシスタントの振る舞いと機能を定義します。私たちはアシスタントに、常にコードの実行とデータベースの推論を通じて回答を検証するよう指示しています。

In [12]:
SYSTEM_PROMPT = あなたは、すべての回答をコード実行によって検証する、役立つAIアシスタントです。

検証の原則:
1. コード、アルゴリズム、または計算について主張する場合は、それを検証するコードを書きます
2. execute_python を使って、数学的な計算、アルゴリズム、論理を検証します
3. 回答を出す前に、理解度を検証するためのテストスクリプトを作成します
4. 実際のコード実行によって作業過程を示します
5. 不確かな場合は、制限を明示的に述べ、検証可能な部分を検証します

アプローチ:
- プログラミングの概念について尋ねられた場合は、コードで実装して説明します
- 計算を求められた場合は、プログラムで計算し、コードを示します
- アルゴリズムを実装する場合は、正しさを証明するためのテストケースを含めます
- 透明性を確保するため、検証プロセスを文書化します
- サンドボックスでは実行間で状態が維持されるため、前の結果を参照できます

利用可能なツール:
- execute_python: Python コードを実行し、出力を確認できます

レスポンス形式: execute_python ツールは、以下の JSON レスポンスを返します:
- sessionId: サンドボックスセッション ID
- id: リクエスト ID
- isError: エラーが発生したかどうかを示すブール値
- content: type と text/data を持つコンテンツオブジェクトの配列
- structuredContent: コード実行の場合、stdout、stderr、exitCode、executionTime が含まれます

## 3. コード実行ツールの定義
次に、エージェントがコードサンドボックスでコードを実行するためのツールとして使用する関数を定義します。@tool デコレータを使って、この関数をエージェントのカスタムツールとしてアノテーションします。

アクティブなコード インタープリター セッション内では、サポートされている言語 (Python、JavaScript) でコードを実行し、依存関係の構成に基づいてライブラリにアクセスし、可視化を生成し、実行間で状態を維持できます。

In [6]:
@tool
def execute_python(code: str, description: str = "") -> str:
    翻訳するテキスト:
"""Execute Python code in the sandbox."""

日本語訳:
"""サンドボックス内で Python コードを実行します。"""
    
    if description:
        code = f"# {description}の説明\n{code}のコード
    
    print(f"\n Generated Code: {code}")
    
    with code_session("us-west-2") as code_client:
        response = code_client.invoke("executeCode", {
            "code": code,
            "language": "python",
            "clearContext": False
        })
    
    for event in response["stream"]:
        return json.dumps(event["result"])

## 4. エージェントの設定
Strands SDK を使用してエージェントを作成し、設定します。上で定義したシステムプロンプトとツールを提供して、コード生成を実行します。

日本語訳:
## 4. エージェントの設定
Strands SDK を使用して エージェント を作成し、設定します。上で定義した system prompt と tool を提供して、コード生成を実行します。

In [9]:
agent = Agent(
    tools=[execute_python],
    system_prompt=SYSTEM_PROMPT,
    callback_handler=None
)

## 5. クエリの定義
コード実行機能を持つエージェントをテストするためのサンプルクエリを定義します。

In [10]:
query = "Tell me the largest random prime number between 1 and 100, which is less than 84 and more that 9"

## 6. エージェントの呼び出しと応答処理
私たちはクエリを使ってエージェントを呼び出し、エージェントの応答を処理します

注意: 非同期実行には非同期環境での実行が必要です

In [11]:
try:
    response_text = ""
    async for event in agent.stream_async(query):
        if "data" in event:
            chunk = event["data"]
            response_text += chunk
            print(chunk, end="")
except Exception as e:
    print(f"Error occurred: {str(e)}")

I'll find the largest random prime number between 1 and 100 that is less than 84 and more than 9. To do this, I'll write code to:

1. Generate all prime numbers in the specified range
2. Filter to keep only those > 9 and < 84
3. Find the largest one

Let me implement this:
 Generated Code: import random

def is_prime(n):
    """Check if a number is prime"""
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

# Find all primes in the range
primes_in_range = [n for n in range(10, 84) if is_prime(n)]

print("All prime numbers between 10 and 83:")
print(primes_in_range)

# Get the largest prime in the range
largest_prime = max(primes_in_range)
print(f"\nThe largest prime number between 10 and 83 is: {largest_prime}")

# For verification, let's check that it's actually prime
print(f"Verifi